In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

pd.set_option('display.max_columns', None)
df = pd.read_csv("/Users/ubokobongudofia/Desktop/X_customer_segmentation/cust_data.csv")

df.head(60)

print(df.loc[df['income'].isnull() | (df['income'] == 0), ['customer_id', 'income']])

df = df[df['gender'] != 'Unknown']

df = df.dropna(subset=['age', 'dw_channel_key', 'income', 'gender'])

cat_vars = ['employment_status', 'marital_status', 'state', 'location']

from sklearn.preprocessing import OrdinalEncoder

# Step 2: Replace missing values with 'Others'
df[cat_vars] = df[cat_vars].fillna('Others')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))

# Half-open bins: [left, right)
bin_edges = [0, 70000, 150000, 300000, 500000, 800000, 1200000, float('inf')]
bin_labels = [
    "<69k",               # strictly less than 100k
    "70k - 149k",         # includes 100k, excludes 200k
    "150k - 299k",         # includes 200k, excludes 400k
    "300k - 499k",         # includes 400k, excludes 1M
    "500k - 799k",           # includes 1M, excludes 5M
    "800k - 1.19M",           # includes 5M, excludes 10M
    "1.2M and above"        # 40M and higher
]

df['income_bracket'] = pd.cut(
    df['income'],
    bins=bin_edges,
    labels=bin_labels,
    right=False,          # left-inclusive, right-exclusive
    include_lowest=True   # includes 0 in the first bin
)

bracket_counts = df['income_bracket'].value_counts().sort_index()

ax = sns.barplot(
    x=bracket_counts.index, 
    y=bracket_counts.values, 
    palette="viridis",
    edgecolor='black'
)

plt.title('Income Distribution', fontsize=14, pad=20)
plt.xlabel('Income Bracket', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)
plt.xticks(rotation=45, ha='right')

total = len(df)
for p in ax.patches:
    height = p.get_height()
    ax.text(
        p.get_x() + p.get_width()/2.,
        height + 0.01*max(bracket_counts),
        f'{int(height):,}\n({height/total:.1%})',
        ha='center',
        va='bottom',
        fontsize=10
    )

sns.despine()
plt.tight_layout()
plt.show()


# Drop all rows where age is greater than 65
df = df[df['age'] <= 65]

# Calculate default rate by age
age_default = df.groupby('age')['has_14plus_dpd'].mean().sort_index()

# Set age range parameters
min_age = 18  
max_age = 65  
target_groups = 4  # This will create 5 groups (n+1 boundaries)

# 1. Create initial bins based on customer distribution percentiles
percentiles = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
age_bins = [int(df['age'].quantile(q)) for q in percentiles]
age_bins = [min_age] + age_bins[1:-1] + [max_age]   # ✅ start at 18 instead of 17

# 2. Adjust bins to ensure minimum group size (no group < 5)
min_group_size = len(df) // target_groups * 0.5  # At least half of the average group size

for i in range(1, len(age_bins)-1):
    lower = age_bins[i-1]
    upper = age_bins[i]
    group_size = ((df['age'] >= lower) & (df['age'] < upper)).sum()

    if group_size < min_group_size:
        # Merge with the previous group if it's too small
        age_bins[i] = age_bins[i-1]

# Remove duplicate bins
age_bins = sorted(list(set(age_bins)))
age_bins = [x for i, x in enumerate(age_bins) if i == 0 or x > age_bins[i-1]]

# 3. Ensure exactly 5 groups by adjusting intervals
while len(age_bins) < target_groups + 1:
    # Find the largest interval and split it
    intervals = [(age_bins[i+1] - age_bins[i], i) for i in range(len(age_bins)-1)]
    max_interval = max(intervals)
    split_point = age_bins[max_interval[1]] + (max_interval[0] // 2)
    age_bins.append(split_point)
    age_bins = sorted(list(set(age_bins)))

# ✅ Ensure bins start at 18 and end at 65 cleanly
age_bins = [min_age] + [x for x in age_bins if min_age < x < max_age] + [max_age]

# 4. Create human-readable labels with clear boundaries
labels = []
for i in range(len(age_bins)-1):
    lower = int(age_bins[i])
    upper = int(age_bins[i+1])
    
    # For human readability, make boundaries non-overlapping
    if i > 0:  # All groups except the first
        lower_bound = lower + 1
    else:
        lower_bound = lower
        
    labels.append(f"{lower_bound}-{upper}")

# Create ordered categories
age_categories = pd.CategoricalDtype(categories=labels, ordered=True)
df['age_category'] = pd.cut(
    df['age'],
    bins=age_bins,
    labels=labels,
    right=True,
    include_lowest=True
).astype(age_categories)

# 5. Visualize and print results
plt.figure(figsize=(10,5))
default_rates = df.groupby('age_category', observed=True)['has_14plus_dpd'].mean()
default_rates.plot(kind='bar', color='skyblue')
plt.title(f"Default Rate by Age Groups (18-65)", pad=20)
plt.ylabel("Default Rate")
plt.xlabel("Age Range")
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nFive Balanced Age Groups:")
for cat in df['age_category'].cat.categories:
    count = (df['age_category'] == cat).sum()
    rate = df[df['age_category'] == cat]['has_14plus_dpd'].mean()
    if count > 0:  # Only show groups with customers
        print(f"{cat} years: {count:,} customers | Default Rate = {rate:.1%}")


import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Data Preparation (exactly as in your original code)
demographic_features = ['gender', 'marital_status', 'purpose', 'employment_status', 'dw_channel_key']
for feature in demographic_features:
    df[feature] = df[feature].astype('category').cat.codes

df[demographic_features].head()


# Create working copy with payment behavior features only
working_df = df[['customer_id', 'total_missed_installment', 'total_amount_overdue', 
                'tenor_in_months', 'total_loan_amount', 'maturity_dpd',
                'has_14plus_dpd', 'count_14plus_dpd', 'ontime_repayment_rate']].copy()

# Payment Behavior
working_df['missed_payment_ratio'] = working_df['total_missed_installment'] / (working_df['tenor_in_months'] + 1e-6)
working_df['overdue_utilization'] = working_df['total_amount_overdue'] / (working_df['total_loan_amount'] + 1e-6)

# Customer Value
working_df['monthly_loan_volume'] = working_df['total_loan_amount'] / (working_df['tenor_in_months'] + 1e-6)
working_df['repayment_efficiency'] = working_df['ontime_repayment_rate'] / 100  # Convert to 0-1 scale



def segment_customer(row):
    # -------- RISK CLASSIFICATION (data-driven) --------
    if (
        row['count_14plus_dpd'] <= 0 and
        row['missed_payment_ratio'] <= 0.4 and
        row['maturity_dpd'] <= 0
    ):
        risk = 'Low'
        
    elif (
        row['count_14plus_dpd'] <= 1 and
        row['missed_payment_ratio'] <= 0.7 and
        row['maturity_dpd'] <= 14
    ):
        risk = 'Medium'
        
    else:
        return 'High Risk'

    value = 'High Value' if (
        row['monthly_loan_volume'] >= working_df['monthly_loan_volume'].quantile(0.75) and  
        row['repayment_efficiency'] >= 0.15                        
    ) else 'Low Value'
    
    return f"{risk} Risk - {value}"


working_df['segment'] = working_df.apply(segment_customer, axis=1)
# Define segment order
segment_order = [
    'Low Risk - High Value',
    'Low Risk - Low Value',
    'Medium Risk - High Value', 
    'Medium Risk - Low Value',
    'High Risk'
]

# Calculate counts and percentages
segment_counts = working_df['segment'].value_counts()[segment_order]
percentages = (segment_counts / len(working_df)) * 100

# Plot the distribution
plt.figure(figsize=(10, 6))
bars = plt.bar(segment_order, segment_counts, color=['#2ecc71', '#27ae60', '#f39c12', '#e67e22', '#e74c3c'])
for bar, count, pct in zip(bars, segment_counts, percentages):
    plt.text(bar.get_x() + bar.get_width()/2, 
             bar.get_height() + 5, 
             f'{count}\n({pct:.1f}%)',
             ha='center', va='bottom', fontsize=10)

plt.title('Customer Segmentation Distribution', fontsize=14, pad=20)
plt.ylabel('Number of Customers', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.ylim(0, max(segment_counts) * 1.15)
sns.despine()
plt.tight_layout()
plt.show()


# 1. Segment order
segment_order = [
    'Low Risk - High Value',
    'Low Risk - Low Value',
    'Medium Risk - High Value', 
    'Medium Risk - Low Value',
    'High Risk'
]

# 2. Define all demographic variables
all_demographics = demographic_features + ['age_category', 'income_bracket']

# 3. Merge demographics into working_df
working_df = working_df.merge(
    df[['customer_id'] + all_demographics],
    on='customer_id',
    how='left'
)

# 4. Calculate population_totals for expected proportions
population_totals = {}
for demo in all_demographics:
    population_totals[demo] = df[demo].value_counts()


df = working_df.dropna(subset=['age_category'])

category_counts = working_df['age_category'].value_counts(dropna=False).sort_index()
print(category_counts)


pd.set_option("display.max_colwidth", None)
import itertools
import pandas as pd

# PARAMETERS (YOUR REQUESTED CHANGES)
OR_THRESHOLD = 1.2        # Keep OR threshold at 1.2
COVERAGE_THRESHOLD = 0.0 # Changed from 0.05 to 0.02 (2%)
MAX_COMBOS_PER_SEGMENT = 2 # Max 2 clusters per segment

def calculate_or(count, total_segment, pop_count, total_pop, eps=1e-6):
    observed = count / total_segment if total_segment > 0 else 0.0
    expected = pop_count / total_pop if total_pop > 0 else 0.0
    OR = observed / (expected + eps)
    return OR, observed, expected

combo_id_counter = 1
final_segment_profiles = {}

for segment in segment_order:
    segment_data = working_df[working_df['segment'] == segment].copy()
    seg_size = len(segment_data)
    total_size = len(df)
    
    print(f"\n{'='*70}")
    print(f"SEGMENT: {segment.upper()}  |  Total Customers: {seg_size}")
    print(f"{'='*70}")
    
    # Step 1: Find qualifying categories (OR ≥ 1.2)
    qualifying_categories = {}
    for demo in all_demographics:
        qualifying_categories[demo] = []
        cat_counts = segment_data[demo].value_counts(dropna=False)

        for value, count in cat_counts.items():
            pop_count = population_totals[demo].get(value, 0)
            OR, obs, exp = calculate_or(count, seg_size, pop_count, total_size)
            if OR >= OR_THRESHOLD:
                qualifying_categories[demo].append((value, OR, obs, exp, count))
    
    features_with_cats = [demo for demo, cats in qualifying_categories.items() if cats]
    if len(features_with_cats) < 2:
        print("Not enough qualifying features to form combinations (need ≥2).")
        continue

    selected_records = []
    
    # Step 2: Try largest combinations first (your original logic)
    for r in range(len(features_with_cats), 1, -1):
        combo_records = []
        
        for feature_subset in itertools.combinations(features_with_cats, r):
            category_options = [
                [(demo, cat[0]) for cat in qualifying_categories[demo]]
                for demo in feature_subset
            ]

            for category_combo in itertools.product(*category_options):
                # Create mask for this combination
                mask = pd.Series(True, index=segment_data.index)
                for demo, value in category_combo:
                    mask &= (segment_data[demo] == value)
                match_count = int(mask.sum())
                
                # Apply 2% coverage threshold (your requested change)
                if match_count / seg_size < COVERAGE_THRESHOLD:
                    continue

                # Calculate population stats
                pop_mask = pd.Series(True, index=df.index)
                for demo, value in category_combo:
                    pop_mask &= (df[demo] == value)
                pop_count = int(pop_mask.sum())

                OR, obs, exp = calculate_or(match_count, seg_size, pop_count, total_size)

                combo_records.append({
                    'Combination Size': r,
                    'Combination ID': f"C{combo_id_counter}",
                    'Combination': " | ".join([f"{demo}: {value}" for demo, value in category_combo]),
                    'Observed Proportion': round(obs, 4),
                    'Expected Proportion': round(exp, 4),
                    'OR': round(OR, 3),
                    'Count': match_count,
                    'Coverage': f"{match_count/seg_size:.1%}"  # Formatted as %
                })
                combo_id_counter += 1

        if combo_records:
            df_output = pd.DataFrame(combo_records)
            # Sort by coverage then OR (your original priority)
            df_output = df_output.sort_values(
                by=["Observed Proportion", "OR"],
                ascending=[False, False]
            ).head(MAX_COMBOS_PER_SEGMENT)  # Your requested max of 2
            
            selected_records = df_output
            break

    if len(selected_records) > 0:
        final_segment_profiles[segment] = selected_records
        print(f"Top {len(selected_records)} clusters for {segment}:")
        display(selected_records[['Combination ID', 'Combination', 'OR', 'Coverage', 'Count']])
    else:
        print(f"No combinations found with OR ≥ {OR_THRESHOLD} AND coverage ≥ {COVERAGE_THRESHOLD:.0%}")